In [13]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from numba import jit
import ipywidgets as wid
from ipywidgets import interactive
import numpy.polynomial.polynomial as poly

RY=13.60566 #x*RY: Rydberg -> ev
ang=1.88972 #X/ang: raio de bohr -> angstroms

def diagg(boo):
    global x,psi,prob,eig,V,L,N
    DIAG.description='Diagonalizando...'
    
    @jit
    def diagH(H,V):
        H=H+V
        eig, psi=np.linalg.eigh(H) #Resolve o sistema e atribui-se: eig=autovalores, psi=autovetores
        prob=np.abs(psi)**2 #Atribui a densidade de probabilidade |Psi|² a variável prob
        return H,V,psi,prob,eig
    
    N=x.size #Armazena o tamanho do vetor posição e o comprimento do espaço que ele representa
    a=- 1/(dx**2) #Define 'a' por conveniência
    H=sparse.diags([a,-2*a,a],[-1,0,1],shape=(N,N),dtype=np.complex128).toarray() #Cria a matriz Hamiltoniano
    H,V,psi,prob,eig=diagH(H,V)
    
    V=np.diag(V)
    PLOT.children[5].disabled=False
    PLOT.children[5].description=('Plotar')
    DIAG.disabled=True
    DIAG.description='Diagonalizado!'
    
def syst(xx,N,potV):   
    global x,dx,V
    def vetores(xx,N):
        xmin,xmax=xx
        x,dx=np.linspace(xmin*ang,xmax*ang, N, 
                         retstep=True,dtype=np.float64) #Define o vetor posição (raio de bohr)
        return x,dx,xmin,xmax
    
    def pot(N,pot):
        V=np.ones(shape=(N))
        V=V*eval(pot)/RY
        V=np.diag(V)
        return V
    
    x, dx, xmin, xmax=vetores(xx,N)
    V=pot(N,potV)
    
    L=(x.max()-x.min())
    
    info.children[1].value=f"{dx/ang: .2f}"
    info.children[3].value=f"{L/ang: .2f}"
    info.children[5].value=f"[{V.min()*RY:.5f} , {V.max()*RY:.5f}]"
    
    #fig1, ax1= plt.subplots(figsize=(12,8))
    #ax1.set_xlabel('$x (\AA)$')
    #ax1.set_ylabel('Potencial (eV)')  
    
    #plt.plot(x/ang,np.diag(V)*RY,'k-',label='$V(x)$')
    #plt.axvline(x=x.min()/ang,color='k',ls='--',label='x mínimo')
    #plt.axvline(x=x.max()/ang,color='k',ls='--',label='x máximo')
    #leg=plt.legend(loc='upper right')
    
    DIAG.disabled=False
    DIAG.description='Diagonalizar'
    PLOT.children[5].disabled=True
    PLOT.children[5].description=('Diagonalize...')
    return V,x,dx,xmin,xmax
    
def plotall(NES,limx,wav,xs,ys):
    fig, ax= plt.subplots(figsize=(xs,ys))
    nmin,nmax=NES
    
    ax.set_xlim([(x.min()-limx)/ang, (x.max()+ limx)/ang])
    ax.set_xlabel('$x (\AA)$')
    ax.set_ylabel('Auto energias (eV)')
    plt.title(f"$\Delta x$ = {dx/ang:.2f}, Ordem da matriz = {N} x {N}")
    
    if wav==1:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,prob[:,i]+eig[i], 
                         label=f'$E_{{{i}}}={eig[i]:.5f}$ eV')
    elif wav==2:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.real(psi[:,i]), 
                         label=f'$E_{{{i}}}={eig[i]:.5f}$ eV')
    elif wav==3:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.real(-1*psi[:,i]), 
                         label=f'$E_{{{i}}}={eig[i]:.5f}$ eV')
    elif wav==4:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.imag(psi[:,i]), 
                         label=f'$E_{{{i}}}={eig[i]:.5f}$ eV')
    
    fig=plt.axvline(x=x.min()/ang,color='k',ls='--')
    fig=plt.axvline(x=x.max()/ang,color='k',ls='--')
    fig=plt.plot(x/ang,V*RY,'k',label='V($x$)')
    leg=plt.legend(loc='upper right')
    plt.show()

################################STYLES/LAYOUT################################
lslider=wid.Layout(width='90%', display='flex')
sty = {'description_width': 'initial'}

################################INTERACTIVES################################
SYST=interactive(syst, {'manual': True}, 
                    xx=wid.FloatRangeSlider(
                        value=[-25.0, 25.0],
                        min=-150.0,
                        max=150.0,
                        step=1,
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='1d',
                        layout=lslider,
                        description=('Domínio de $\Psi(x)$: '),
                        style=sty),
                    N=wid.IntSlider(value=500,
                        min=50,
                        max=1000,
                        step=10,
                        continuous_update=False, 
                        layout=lslider,
                        description=('Número de pontos: '),
                        style=sty),
                    potV=wid.Text(description='Expressão de $V(x)$: ',
                        value="0.00001*(x**2)",
                        placeholder='Escreva a expressão aqui.',
                        style=sty,
                        layout=lslider,
                        continuous_update=False))

PLOT=interactive(plotall, {'manual': True},
                    NES=wid.IntRangeSlider(
                        value=[0, 3],
                        min=0,
                        max=20,
                        step=1,
                        description='Plotar estados:',
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='.1d',
                        layout=lslider,
                        style=sty),
                    limx=wid.IntSlider(
                        value=20,
                        min=0,
                        max=80,
                        step=1,
                        description='Margens horizontais em $x$:   ',
                        continuous_update=False,
                        layout=lslider,
                        style=sty),
                    wav=wid.Dropdown(
                        options=[('densidades de probabilidades',1),
                                 ('Funções de onda (parte real, +)',2),
                                 ('Funções de onda (parte real, -)',3),
                                 ('Funções de onda (parte imaginária)',4)],
                        value=1,
                        description='Plotar as',
                        continuous_update=True,
                        style=sty),
                    xs=wid.BoundedIntText(
                        value=16,
                        min=0,
                        max=20,
                        step=1,
                        description='Tamanho da figura (X):',
                        continuous_update=True,
                        style=sty),
                    ys=wid.BoundedIntText(
                        value=9,
                        min=0,
                        max=20,
                        step=1,
                        description='Tamanho da figura (Y):',
                        continuous_update=True,
                        style=sty))

################################BOTOES################################
DIAG=wid.Button(description='Diagonalizar',disabled=True)

################################GRIDS################################
items = [wid.Label('Passo $\Delta x $ (angstroms)='), wid.Label('dx'), 
        wid.Label('Largura do domínio (angstroms) ='), wid.Label('L'),
        wid.Label('[V_min , V_max] (eV) ='), wid.Label('[V_min,V_max]'),]
info=wid.GridBox(items, layout=wid.Layout(grid_template_columns="repeat(2, 250px)"))

################################ON_EVENTS###############################
DIAG.on_click(diagg)

################################UI################################
SYST.children[3].description=('Gerar');
PLOT.children[5].description=('Plotar');
PLOT.children[5].disabled=True
uip=wid.VBox([SYST,info,DIAG])

################################TABS################################
tab=wid.Tab(children=[uip,PLOT])
tab.set_title(0, 'Gerar o Sistema')
tab.set_title(1, 'Funções de Onda')

display(tab)

***
# Como usar

* 1. Execute o código acima.

* 2. Na aba 'Gerar o Sistema' escolha os parâmetros e clique em 'Gerar' para gerar a equação matricial.
    * Domínio de $\Psi(x)$: o intervalo fechado $X=[x_{min},x_{max}]$ onde a função de onda será descrita.
    * Número de pontos: o número de pontos que serão usados para descrever a função de onda.
    
* 3. Clique em 'Diagonalizar' para resolver o sistema matricial. 

* 4. Na aba 'Funções de Onda' escolha os parâmetros do plot e clique em 'Plotar'.

***